# Understands aims_ss, aims_bottom and aims_other from .arg graph

In [4]:
import sys
import os
import json
from os.path import join
import numpy as np
import pandas as pd

# Import the aims module
from soma import aims
# the brainplot package
import colorado as cld

print(sys.version)


3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]


In [5]:
sys.path.append((os.path.abspath('../')))
import deep_folding
print(os.path.dirname(deep_folding.__file__))

/home/jc225751/Program/deep_folding/deep_folding


# User variables

In [6]:
src_dir = join(os.getcwd(), 
                       '../data/source/unsupervised',
                       'ANALYSIS/3T_morphologist/100206/t1mri',
                       'default_acquisition/default_analysis')
src_dir = os.path.abspath(src_dir)
print("src_dir = " + src_dir)

src_dir = /home/jc225751/Program/deep_folding/data/source/unsupervised/ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis


In [7]:
graph = aims.read(join(src_dir, 'folds/3.1/default_session_auto', 'L100206_default_session_auto.arg'))
skeleton = aims.read(join(src_dir, 'segmentation', 'Lskeleton_100206.nii.gz'))

In [9]:
dict_bottom = {}
dict_ss = {}
dict_other = {}
none_number = 0
multiple_number = {0: 0, 1: 0, 2: 0, 3: 0}
number_vertices = 0
number_voxels = 0

def add_value_to_dict(vertex, bucket_name, dictionary):
    global number_voxels
    bucket = vertex.get(bucket_name)
    if bucket is not None:
        for voxel in bucket[0].keys():
            value = skeleton[voxel[0], voxel[1], voxel[2]]
            str_value = str(value)
            if str_value not in dictionary.keys():
                dictionary[str_value] = 1
            dictionary[str_value] += 1
            number_voxels += 1
    return bucket

def number_true(iterable):
    it = iter(iterable)
    trues = [bool(i) for i in it]
    return trues.count(True)    
    
for vertex in graph.vertices():
    vname = vertex.get('name')
 #   for bucket_name in ('aims_ss', 'aims_bottom', 'aims_other'):
    bck_present = []
    bck_present.append(add_value_to_dict(vertex, 'aims_bottom', dict_bottom))
    bck_present.append(add_value_to_dict(vertex, 'aims_ss', dict_ss))
    bck_present.append(add_value_to_dict(vertex, 'aims_other', dict_other))
    if number_true(bck_present) == 0:
        none_number += 1
    
    multiple_number[number_true(bck_present)] += 1
    
    number_vertices +=  1

print('Topology values corresponding to')
print('aims_bottom : ', dict_bottom)
print('aims_ss : ', dict_ss)
print('aims_other : ', dict_other)
print("vertices with aims surface type undefined : ", none_number)
print("vertices with several aims surface types defined : ", multiple_number)
print('')
print("number of vertices", number_vertices)
print("number of voxels", number_voxels)

Topology values corresponding to
aims_bottom :  {'[30]': 8857}
aims_ss :  {'[60]': 76037}
aims_other :  {'[80]': 1727, '[30]': 639, '[60]': 1066, '[40]': 80, '[10]': 7}
vertices with aims surface type undefined :  1
vertices with several aims surface types defined :  {0: 1, 1: 0, 2: 0, 3: 407}

number of vertices 408
number of voxels 88406


In [10]:
arr_skel = skeleton.arraydata()
np.unique(arr_skel)

array([ 0, 10, 11, 30, 40, 60, 80], dtype=int16)

In [11]:
pd.value_counts(np.resize(arr_skel, arr_skel.size))

11    18832738
0      2012736
60      163069
30        9634
80        5330
40          87
10           6
dtype: int64

In [12]:
np.count_nonzero(arr_skel > 11) + np.count_nonzero(arr_skel == 10)

178126

In [14]:
voxel_size = graph['voxel_size'][:3]
print(voxel_size)

[0.699999988079071, 0.7000120282173157, 0.7000120282173157]


In [16]:
print(skeleton.getVoxelSize())

[0.700012, 0.700012, 0.700012, 1]


In [23]:
print(skeleton)
print(skeleton.shape)

(1, 260, 311, 260)


In [24]:
print(skeleton[0,1,1,1])
print(arr_skel[0,1,1,1])

11
11


In [25]:
print(arr_skel.shape)

(1, 260, 311, 260)


In [27]:
print(skeleton.getSize())

[260, 311, 260, 1]
